In [1]:
import torch 
import pandas as pd
from sklearn.preprocessing import OneHotEncoder as ohe
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm


In [2]:
df = pd.read_csv("../../data/raw/obesity_prediction.csv")

In [3]:
df.head(3)

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Obesity
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight


In [4]:
enc = ohe(handle_unknown="warn",sparse_output=False).set_output(transform="pandas")

In [5]:
enc.fit(np.asarray(df["Gender"]).reshape(-1,1))

OneHotEncoder(handle_unknown='warn', sparse_output=False)

In [6]:
enc.categories_

[array(['Female', 'Male'], dtype=object)]

In [7]:
gender_enc_df = enc.transform(np.asarray(df["Gender"]).reshape(-1,1))

In [8]:
gender_enc_df

,x0_Female,x0_Male
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
2106,1.0,0.0
2107,1.0,0.0
2108,1.0,0.0
2109,1.0,0.0


In [9]:
gender_enc_df.columns =["female","male"]

In [10]:
gender_enc_df.head()

,female,male
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [11]:
enc2 = ohe(sparse_output=False,handle_unknown="ignore").set_output(transform="pandas")

In [12]:
df_mtran = enc2.fit_transform(np.asarray(df["MTRANS"]).reshape(-1,1))

In [13]:
df_mtran.head()

,x0_Automobile,x0_Bike,x0_Motorbike,x0_Public_Transportation,x0_Walking
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0


In [14]:
enc2.categories_

[array(['Automobile', 'Bike', 'Motorbike', 'Public_Transportation',
        'Walking'], dtype=object)]

In [15]:
df_mtran.columns = ["automobile","bike","motorbike","public_transportation","walking"]

In [16]:
df_mtran.head(3)

,automobile,bike,motorbike,public_transportation,walking
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0


In [17]:
df_mtran.head()

,automobile,bike,motorbike,public_transportation,walking
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0


In [18]:
gender_enc_df.head()

,female,male
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [19]:
df_x = pd.concat(objs=[gender_enc_df,df_mtran,df["Age"],df["Height"]],axis=1)

In [20]:
df_x

,female,male,automobile,bike,motorbike,public_transportation,walking,Age,Height
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,21.000000,1.620000
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,21.000000,1.520000
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,23.000000,1.800000
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,27.000000,1.800000
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,22.000000,1.780000
...,...,...,...,...,...,...,...,...,...
2106,1.0,0.0,0.0,0.0,0.0,1.0,0.0,20.976842,1.710730
2107,1.0,0.0,0.0,0.0,0.0,1.0,0.0,21.982942,1.748584
2108,1.0,0.0,0.0,0.0,0.0,1.0,0.0,22.524036,1.752206
2109,1.0,0.0,0.0,0.0,0.0,1.0,0.0,24.361936,1.739450


In [21]:
from sklearn.preprocessing import LabelEncoder

In [22]:
le = LabelEncoder()

In [23]:
y_enc = le.fit_transform(df["Obesity"])

In [24]:
y_enc

array([1, 1, 1, ..., 4, 4, 4], shape=(2111,))

In [25]:
le.classes_

array(['Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_I',
       'Obesity_Type_II', 'Obesity_Type_III', 'Overweight_Level_I',
       'Overweight_Level_II'], dtype=object)

In [26]:
Y = torch.tensor(y_enc)

In [27]:
Y.shape

torch.Size([2111])

In [28]:
scaler = StandardScaler().set_output(transform="pandas")

In [29]:
df_age_scaled = scaler.fit_transform(np.asarray(df["Age"]).reshape(-1,1))
df_height_scaled = scaler.fit_transform(np.asarray(df["Height"]).reshape(-1,1))

In [30]:
df_height_scaled.columns =["height"]
df_age_scaled.columns = ["age"]

In [31]:
df_height_scaled

,height
0,-0.875589
1,-1.947599
2,1.054029
3,1.054029
4,0.839627
...,...
2106,0.097045
2107,0.502844
2108,0.541672
2109,0.404927


In [32]:
df_x = df_x.drop(labels=["Age","Height"],axis=1)
df_x.head()

,female,male,automobile,bike,motorbike,public_transportation,walking
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [33]:
df_x = pd.concat(axis=1,objs=[df_x,df_age_scaled,df_height_scaled])

In [34]:
df_x.head()

,female,male,automobile,bike,motorbike,public_transportation,walking,age,height
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.522124,-0.875589
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.522124,-1.947599
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.206889,1.054029
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.423582,1.054029
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.364507,0.839627


In [35]:
class ffn(torch.nn.Module):
    
    def __init__(self,input_dims):
        super(ffn,self).__init__()    
        
        self.relu    = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        self.tanh    = torch.nn.Tanh()
        self.layer1  = torch.nn.Linear(input_dims,70)
        self.layer2  = torch.nn.Linear(70,70)
        self.layer3  = torch.nn.Linear(70,30)
        self.layer4  = torch.nn.Linear(30,7)
    def forward(self,input):
        
        out = self.tanh(self.layer1(input))
        out = self.tanh(self.layer2(out))
        out = self.tanh(self.layer3(out))
        out = self.layer4(out)
        return out

In [36]:
Y

tensor([1, 1, 1,  ..., 4, 4, 4])

In [37]:
X = torch.tensor(np.asarray(df_x))

In [38]:
X.shape

torch.Size([2111, 9])

In [39]:
df_x.head()

,female,male,automobile,bike,motorbike,public_transportation,walking,age,height
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.522124,-0.875589
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.522124,-1.947599
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.206889,1.054029
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.423582,1.054029
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.364507,0.839627


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [41]:
class data(torch.utils.data.Dataset):

    def __init__(self, X,y):
        self.x = X
        self.y = y
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        record = self.x[index]
        label = self.y[index]
        return record,label

In [42]:
len(X_train[0])

9

In [43]:
model = ffn(9)

In [44]:
train_dataset = data(X_train,y_train)
test_dataset = data(X_test,y_test)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=32,shuffle=True)

In [ ]:
model.train()
epochs = 200

#optim = torch.optim.SGD(model.parameters(),lr = 0.1,momentum=.7)
optim = torch.optim.AdamW(model.parameters(),lr = 0.001)
loss_fn = torch.nn.CrossEntropyLoss()
y_true = []
y_pred = []

for epoch in range(epochs):
    loss_per_epoch = 0
    for batch in tqdm(train_loader):
        
        inputs,labels = batch
        optim.zero_grad()
        
        outputs = model(inputs.float())
        loss    = loss_fn(outputs,labels)
       
        _,preds = torch.max(outputs.detach(),1)
    
        y_pred.extend(preds.detach().tolist())
        y_true.extend(labels.detach().tolist())
        loss.backward()
        optim.step()
        
        loss_per_epoch = loss.item()
    
    print(f"Epoch: {epoch + 1}   Loss: {loss_per_epoch}")
    print(classification_report(y_true,y_pred,zero_division=0))

100%|██████████| 45/45 [00:00<00:00, 843.55it/s]


Epoch: 1   Loss: 1.7921299934387207
              precision    recall  f1-score   support

           0       0.26      0.07      0.11       182
           1       0.18      0.02      0.04       184
           2       0.18      0.55      0.27       237
           3       0.39      0.50      0.44       203
           4       0.37      0.57      0.45       209
           5       0.00      0.00      0.00       196
           6       0.22      0.05      0.09       203

    accuracy                           0.27      1414
   macro avg       0.23      0.25      0.20      1414
weighted avg       0.23      0.27      0.21      1414



100%|██████████| 45/45 [00:00<00:00, 761.79it/s]


Epoch: 2   Loss: 1.547091007232666
              precision    recall  f1-score   support

           0       0.30      0.30      0.30       364
           1       0.26      0.06      0.10       368
           2       0.19      0.31      0.24       474
           3       0.38      0.70      0.50       406
           4       0.44      0.78      0.56       418
           5       0.00      0.00      0.00       392
           6       0.28      0.08      0.12       406

    accuracy                           0.33      2828
   macro avg       0.26      0.32      0.26      2828
weighted avg       0.26      0.33      0.26      2828



100%|██████████| 45/45 [00:00<00:00, 860.36it/s]


...

100%|██████████| 45/45 [00:00<00:00, 902.95it/s]

Epoch: 149   Loss: 1.0077452659606934


              precision    recall  f1-score   support

           0       0.52      0.63      0.57     27118
           1       0.48      0.28      0.36     27416
           2       0.54      0.59      0.56     35313
           3       0.67      0.90      0.77     30247
           4       0.73      0.95      0.82     31141
           5       0.54      0.38      0.44     29204
           6       0.59      0.42      0.49     30247

    accuracy                           0.60    210686
   macro avg       0.58      0.59      0.57    210686
weighted avg       0.58      0.60      0.58    210686



100%|██████████| 45/45 [00:00<00:00, 901.78it/s]


Epoch: 150   Loss: 0.346850723028183
              precision    recall  f1-score   support

           0       0.52      0.63      0.57     27300
           1       0.48      0.28      0.36     27600
           2       0.54      0.59      0.57     35550
           3       0.67      0.90      0.77     30450
           4       0.73      0.95      0.82     31350
           5       0.54      0.38      0.45     29400
           6       0.59      0.42      0.49     30450

    accuracy                           0.60    212100
   macro avg       0.58      0.59      0.57    212100
weighted avg       0.58      0.60      0.58    212100



In [46]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    test_loss = 0
    for batch in test_loader:
        
        inputs,labels = batch
        outputs = model(inputs.float())
        loss = loss_fn(outputs,labels)
        test_loss = loss.item()
        _,preds = torch.max(outputs.detach(),1)
        y_pred.extend(preds.detach().tolist())
        y_true.extend(labels.detach().tolist())

    print(f"Test Loss: {test_loss}")
    print(classification_report(y_true,y_pred,zero_division=0))
    
    

Test Loss: 0.6612717509269714
              precision    recall  f1-score   support

           0       0.48      0.64      0.55        90
           1       0.52      0.22      0.31       103
           2       0.53      0.56      0.54       114
           3       0.72      0.84      0.77        94
           4       0.78      0.96      0.86       115
           5       0.48      0.43      0.45        94
           6       0.40      0.36      0.38        87

    accuracy                           0.58       697
   macro avg       0.56      0.57      0.55       697
weighted avg       0.57      0.58      0.56       697



In [47]:
dict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0}

for cls in y_train:

    y = cls.item()

    dict[y] += 1